# Popular Data Science Questions on Stack Exchange

## Introduction

I work for an imaginary company that provides data science content in different formats including books, videos, online articles, or text-based interactive platforms like [Dataquest](http://dataquest.io/).

My task in this project is to find out **'what people want to learn about in data science'**.

To do that, I'll take a leaf from my experience with using [Stack Overflow](https://stackoverflow.com/), a question/answer platform on programming. 

However, Stack Overflow is only a subset of the parent website known as Stack Exchange; which hosts several sites as well as a whole site dedicated to data science questions and answers.

## Background

Stack Exchange is home to a number of sites across various fields including philosophy, physics, mathematics and data science.

(*See a list of sites hosted on Stack Exchange and their performance [here](https://stackexchange.com/sites?view=list#traffic). They've been sorted by the percentage of questions that have received answers.*)

**Kinds of questinons that can be asked...**

The [Data Science Stack Exchange](https://datascience.stackexchange.com/questions)(DSSE), for example, welcomes questions that relate to data science: from tensorflow to neural networks, machine learning, deep learning and much more as long as the questions are:
- Specific to the site's field of expertise.
- Objective rather than subjective.
- Clear and concise.
- Easy for other users to understand.

DSE is currently ranked at the bottom 10 based on the perecentage of answered questions.

**Information available in each post**

The image below is a snippet of [the question with the most upvotes](https://datascience.stackexchange.com/questions/6107/what-are-deconvolutional-layers) on the Data Science Stack Exchange as of August 2020.


!['title'](https://app.dataquest.io/jupyter/files/notebook/Untitled%20Folder/DSSS%20question%20with%20most%20upvotes.PNG)

Some of the features to take note of include the question asked, the number of answers provided, number of times the question post was viewed, the date the question was asked, related tags, as well as information about the author's activity.

Typically, Stack Exchange uses a reputation award system for question asked and answers provided on the different topics. Each question post can also be upvoted or downvoted to make good topics identifiable.

From the snippet, the author, Martin Thoma, has a reputation score of about 14,600 based on the number of times other users upvoted his questions, answers, and edits.

## Stack Exchange Data Explorer

After some investigation on the structure of questions asked, I realized that the `tags` will be quite useful in categorizing content.

Although my first thought was to scrape the site, I found an easier to get this data -- the [Data Science Stack Exchange database](https://data.stackexchange.com/datascience/query/new)!

The image below shows the names of the tables in the database.

!['title'](https://dq-content.s3.amazonaws.com/469/dsde.png)

From this schema, the tables that will be relevant to this project are:
- Posts
- PostTags
- Tags
- Tagsynonyms

From my explorations, I observed that the bulk of my analysis will be focused on the `Posts` table. 

The `Tags` table will also be useful because it shows the number of times a particular tag was used. Unfortunately, it doesn't say anything about how recent the data on the tags was gathered.

## Getting the Data

The `Posts` table has quite a number of columns. for the purpose of this project, I'll focus on the following relevant columns:
- `Id`: A unique identification number for the post.
- `PostTypeId`: A unique identification number for the type of post.
!['title'](https://dq-content.s3.amazonaws.com/469/PostTypes.png)

- `CreationDate`: The date and time the post was created.
- `Score`: The post's score.
- `ViewCount`: The number of times the post was viewed.
- `Tags`: Tags that were used.
- `AnswerCount`: Number of answers that the post got.
- `FavoriteCount`: Number of times the post was liked.

**Takeaways:**
1. There are 8 different kinds of posts based on the `PostTypeId` Column. However, in this project, I'll be working with `Question` posts with `PostTypeId` of 1.

2. The last two columns are particularly useful to my analyis because they have the kind of information that I want.



**Query to select the data from DSSE database:**

`SELECT Id, CreationDate,
       Score, ViewCount, Tags,
       AnswerCount, FavoriteCount
  FROM posts
 WHERE PostTypeId = 1 AND YEAR(CreationDate) = 2019;`
 
**This is what the data looks like:**
!['title'](https://app.dataquest.io/jupyter/files/notebook/data_DSSE%20database.PNG)

## Exploring the Data

The result of the data retrieved from the DSSE database was saved in a file known as `2019_questions.csv`.

I'll be reading the file into Pandas for further exploration of the data.

In [1]:
import pandas as pd
question_posts = pd.read_csv("2019_questions.csv", parse_dates=["CreationDate"])
question_posts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8839 entries, 0 to 8838
Data columns (total 7 columns):
Id               8839 non-null int64
CreationDate     8839 non-null datetime64[ns]
Score            8839 non-null int64
ViewCount        8839 non-null int64
Tags             8839 non-null object
AnswerCount      8839 non-null int64
FavoriteCount    1407 non-null float64
dtypes: datetime64[ns](1), float64(1), int64(4), object(1)
memory usage: 483.5+ KB


**Observation:**
- Only the `FavoriteCount` column has missing values.
- The probable reason `FavoriteCount` has missing values is that the post doesn't show up on any user's favorites list. So, one solution is to fill the missing values with 0.
- The data type of each column seems adequate except that `Tags` has a general rather than specific description. Let's get specific...

In [2]:
question_posts["Tags"].apply(lambda value: type(value)).unique()

array([<class 'str'>], dtype=object)

It is now clear that the `Tags` column has a string type.

## Cleaning the Data

In this section, I will transform the string values of the `Tags` column 

from this:
`<machine-learning><regression><linear-regression><regularization>`

to this:
`machine-learning,regression,linear-regression,regularization`

In [3]:
#Filling the missing values in FavoriteCount column
question_posts["FavoriteCount"] = question_posts["FavoriteCount"].fillna(0).astype(int)

In [4]:
question_posts["Tags"] = question_posts["Tags"].str.replace("^<|>$", "").str.split("><")


In [5]:
question_posts.head(5)

,Id,CreationDate,Score,ViewCount,Tags,AnswerCount,FavoriteCount
0,44419,2019-01-23 09:21:13,1,21,"[machine-learning, data-mining]",0,0
1,44420,2019-01-23 09:34:01,0,25,"[machine-learning, regression, linear-regressi...",0,0
2,44423,2019-01-23 09:58:41,2,1651,"[python, time-series, forecast, forecasting]",0,0
3,44427,2019-01-23 10:57:09,0,55,"[machine-learning, scikit-learn, pca]",1,0
4,44428,2019-01-23 11:02:15,0,19,"[dataset, bigdata, data, speech-to-text]",0,0


In [6]:
tag_count = dict()

for lists in question_posts["Tags"]:
    for tag in lists:
        if tag in tag_count:
            tag_count[tag] += 1
        else:
            tag_count[tag] = 1



In [7]:
#Transforming tag_count dictionary to a Dataframe
tag_count = pd.DataFrame.from_dict(tag_count, orient="index")
tag_count.rename(columns={0: "Count"}, inplace=True)
tag_count.sort_values(by= "Count").tail(10)


,Count
cnn,489
nlp,493
scikit-learn,540
tensorflow,584
classification,685
keras,935
neural-network,1055
deep-learning,1220
python,1814
machine-learning,2693


## Most Used and Most Viewed Tags

In [8]:
tag_count = dict()

for tags in question_posts["Tags"]:
    for tag in tags:
        if tag in tag_count:
            tag_count[tag] += 1
        else:
            tag_count[tag] = 1


`tag_counts` will look better as a dataframe. Next, I'll trnasform it.

In [9]:
tag_count = pd.DataFrame.from_dict(tag_count, orient="index")
tag_count.rename(columns={0: "Count"}, inplace=True)
tag_count.head(10)


,Count
outlier,48
yolo,21
anomaly,4
haar-cascade,1
multi-output,7
methods,4
auc,3
randomized-algorithms,6
automatic-summarization,10
statistics,234


Let's now sort for the most used tags

In [10]:
most_used = tag_count.sort_values(by="Count").tail(20)
most_used

,Count
machine-learning-model,224
statistics,234
clustering,257
predictive-modeling,265
r,268
dataset,340
regression,347
pandas,354
lstm,402
time-series,466


Even though this data doesn't appear overwhelming, representing it on a suitable visualization will be a lot better.

I'll be using a horizontal bar chart to represent the `most_viewed` tags.

In [13]:
import matplotlib as plt
most_used.plot(kind="barh", figsize=(12,6))
plt.show()


AttributeError: 'module' object has no attribute 'show'